# Part 3: Production-Ready Agentic RAG

## Taking Your Agent to Production

In Parts 1 and 2, we built:
- ✅ **Baseline RAG**: Simple retrieval + generation
- ✅ **Agentic RAG**: Planning + tools + citations

But production deployments require:
- 💰 **Cost Control**: Caching, cheaper models
- ⚡ **Performance**: Parallel execution, latency optimization
- 📊 **Observability**: Metrics, logging, tracing
- 🛡️ **Safety**: Guardrails, PII protection, validation
- 🚀 **Deployment**: API wrapper, scaling, monitoring

---

## What We'll Build

1. **Caching Layer**: Semantic + exact match caching
2. **Observability**: Detailed metrics and tracing
3. **Safety Guardrails**: PII masking, cost limits, validation
4. **Production API**: FastAPI deployment wrapper

Let's make our agent production-ready! 🚀

---

## Setup and Imports

In [1]:
import os
import json
import time
import hashlib
import re
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
from dataclasses import dataclass
from collections import defaultdict
import httpx
from openai import OpenAI
from pydantic import BaseModel, Field
import numpy as np

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# ⚠️ SSL BYPASS FOR CORPORATE NETWORKS (DEMO ONLY - NOT FOR PRODUCTION)
http_client = httpx.Client(verify=False)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=http_client
)

print("✅ Environment loaded")

✅ Environment loaded


---

## 1. Caching Layer

### Why Caching Matters

**Problem**: Every query costs money and time
- Embedding call: ~$0.0001
- LLM call: ~$0.01-0.10
- Tool calls: Variable latency

**Solution**: Cache at multiple levels
1. **Exact Match Cache**: Identical queries (hash-based)
2. **Semantic Cache**: Similar queries (embedding-based)
3. **Tool Result Cache**: Repeated API calls

### Implementation

In [2]:
@dataclass
class CacheEntry:
    """Cache entry with metadata."""
    query: str
    response: str
    embedding: np.ndarray
    timestamp: datetime
    hits: int = 0
    cost_saved: float = 0.0


class SemanticCache:
    """Multi-level caching for agent responses."""
    
    def __init__(self, similarity_threshold: float = 0.95, ttl_hours: int = 24):
        self.exact_cache: Dict[str, CacheEntry] = {}  # Hash -> Entry
        self.semantic_entries: List[CacheEntry] = []  # For similarity search
        self.similarity_threshold = similarity_threshold
        self.ttl = timedelta(hours=ttl_hours)
        self.stats = {
            "exact_hits": 0,
            "semantic_hits": 0,
            "misses": 0,
            "total_cost_saved": 0.0
        }
    
    def _hash_query(self, query: str) -> str:
        """Generate hash for exact match."""
        return hashlib.sha256(query.lower().strip().encode()).hexdigest()
    
    def _is_expired(self, entry: CacheEntry) -> bool:
        """Check if cache entry is expired."""
        return datetime.now() - entry.timestamp > self.ttl
    
    def _get_embedding(self, text: str) -> np.ndarray:
        """Get embedding for semantic similarity."""
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"  # Cheaper for cache lookups
        )
        return np.array(response.data[0].embedding)
    
    def _cosine_similarity(self, a: np.ndarray, b: np.ndarray) -> float:
        """Calculate cosine similarity."""
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    def get(self, query: str, estimated_cost: float = 0.05) -> Optional[str]:
        """Try to retrieve from cache."""
        # Step 1: Exact match (instant)
        query_hash = self._hash_query(query)
        if query_hash in self.exact_cache:
            entry = self.exact_cache[query_hash]
            if not self._is_expired(entry):
                entry.hits += 1
                entry.cost_saved += estimated_cost
                self.stats["exact_hits"] += 1
                self.stats["total_cost_saved"] += estimated_cost
                print(f"💾 [CACHE HIT - EXACT] Saved ${estimated_cost:.4f}")
                return entry.response
        
        # Step 2: Semantic similarity (slower, but catches paraphrases)
        query_embedding = self._get_embedding(query)
        
        for entry in self.semantic_entries:
            if self._is_expired(entry):
                continue
            
            similarity = self._cosine_similarity(query_embedding, entry.embedding)
            if similarity >= self.similarity_threshold:
                entry.hits += 1
                entry.cost_saved += estimated_cost
                self.stats["semantic_hits"] += 1
                self.stats["total_cost_saved"] += estimated_cost
                print(f"💾 [CACHE HIT - SEMANTIC] Similarity: {similarity:.3f}, Saved ${estimated_cost:.4f}")
                return entry.response
        
        # Miss
        self.stats["misses"] += 1
        return None
    
    def set(self, query: str, response: str):
        """Store in cache."""
        query_hash = self._hash_query(query)
        embedding = self._get_embedding(query)
        
        entry = CacheEntry(
            query=query,
            response=response,
            embedding=embedding,
            timestamp=datetime.now()
        )
        
        self.exact_cache[query_hash] = entry
        self.semantic_entries.append(entry)
    
    def get_stats(self) -> Dict[str, Any]:
        """Get cache statistics."""
        total_queries = sum([self.stats["exact_hits"], self.stats["semantic_hits"], self.stats["misses"]])
        hit_rate = (self.stats["exact_hits"] + self.stats["semantic_hits"]) / total_queries if total_queries > 0 else 0
        
        return {
            **self.stats,
            "total_queries": total_queries,
            "hit_rate": hit_rate,
            "cache_size": len(self.exact_cache)
        }

# Initialize cache
cache = SemanticCache(similarity_threshold=0.95, ttl_hours=24)
print("✅ Semantic cache initialized")

✅ Semantic cache initialized


### Test Caching

In [3]:
# Demo: Cache behavior
print("Testing cache...\n")

# First query - MISS
result1 = cache.get("Where is my order #12345?")
print(f"First query result: {result1}\n")

# Store result
cache.set("Where is my order #12345?", "Your order #12345 is in transit...")

# Second query - EXACT HIT
result2 = cache.get("Where is my order #12345?")
print(f"Second query result: {result2[:50]}...\n")

# Third query - SEMANTIC HIT (paraphrase)
result3 = cache.get("What's the status of order 12345?")
print(f"Paraphrased query result: {result3[:50] if result3 else 'MISS (threshold not met)'}...\n")

# Show stats
print("Cache Statistics:")
print(json.dumps(cache.get_stats(), indent=2))

Testing cache...

First query result: None

💾 [CACHE HIT - EXACT] Saved $0.0500
Second query result: Your order #12345 is in transit......

Paraphrased query result: MISS (threshold not met)...

Cache Statistics:
{
  "exact_hits": 1,
  "semantic_hits": 0,
  "misses": 2,
  "total_cost_saved": 0.05,
  "total_queries": 3,
  "hit_rate": 0.3333333333333333,
  "cache_size": 1
}


---

## 2. Observability and Metrics

### What to Track

**Key Metrics**:
1. **Latency**: Total time, per-tool time
2. **Cost**: Per query, per tool
3. **Quality**: Citation coverage, retrieval precision
4. **Errors**: Failed tools, rate limits

### Implementation

In [4]:
class AgentMetrics:
    """Track and log agent execution metrics."""
    
    def __init__(self):
        self.query_log: List[Dict[str, Any]] = []
        self.tool_latencies = defaultdict(list)
        self.tool_errors = defaultdict(int)
        
    def log_query(self, query_data: Dict[str, Any]):
        """Log a complete query execution."""
        self.query_log.append({
            **query_data,
            "timestamp": datetime.now().isoformat()
        })
    
    def log_tool_call(self, tool_name: str, latency: float, success: bool):
        """Log individual tool execution."""
        self.tool_latencies[tool_name].append(latency)
        if not success:
            self.tool_errors[tool_name] += 1
    
    def get_summary(self) -> Dict[str, Any]:
        """Get aggregated metrics."""
        if not self.query_log:
            return {"message": "No queries logged yet"}
        
        total_queries = len(self.query_log)
        avg_latency = np.mean([q.get("total_latency", 0) for q in self.query_log])
        avg_cost = np.mean([q.get("estimated_cost", 0) for q in self.query_log])
        avg_steps = np.mean([q.get("steps", 0) for q in self.query_log])
        
        # Tool stats
        tool_stats = {}
        for tool, latencies in self.tool_latencies.items():
            tool_stats[tool] = {
                "calls": len(latencies),
                "avg_latency_ms": np.mean(latencies) * 1000,
                "p95_latency_ms": np.percentile(latencies, 95) * 1000 if len(latencies) > 1 else latencies[0] * 1000,
                "errors": self.tool_errors.get(tool, 0)
            }
        
        return {
            "total_queries": total_queries,
            "avg_latency_sec": round(avg_latency, 3),
            "avg_cost_usd": round(avg_cost, 4),
            "avg_steps": round(avg_steps, 2),
            "tool_stats": tool_stats
        }
    
    def print_trace(self, query_index: int = -1):
        """Pretty print execution trace for a query."""
        if not self.query_log:
            print("No queries to trace")
            return
        
        query = self.query_log[query_index]
        
        print("\n" + "="*80)
        print("EXECUTION TRACE")
        print("="*80)
        print(f"Query: {query.get('query', 'N/A')}")
        print(f"Timestamp: {query.get('timestamp', 'N/A')}")
        print(f"Total Latency: {query.get('total_latency', 0):.3f}s")
        print(f"Estimated Cost: ${query.get('estimated_cost', 0):.4f}")
        print(f"Steps: {query.get('steps', 0)}")
        print(f"Citations: {query.get('citation_count', 0)}")
        
        if "tool_calls" in query:
            print(f"\nTool Calls ({len(query['tool_calls'])})")
            for i, tool_call in enumerate(query['tool_calls'], 1):
                print(f"  {i}. {tool_call['name']}({tool_call.get('args', {})})")
                print(f"     → Latency: {tool_call.get('latency', 0):.3f}s")
                print(f"     → Status: {'✅ Success' if tool_call.get('success', True) else '❌ Failed'}")
        
        print("="*80)

# Initialize metrics
metrics = AgentMetrics()
print("✅ Metrics tracker initialized")

✅ Metrics tracker initialized


---

## 3. Safety Guardrails

### Why Guardrails Matter

**Risks**:
- Infinite loops → Cost explosion
- PII leakage → Compliance violations
- Hallucinations → Customer trust loss
- API abuse → Rate limiting

### Implementation

In [5]:
class SafetyGuardrails:
    """Production safety controls."""
    
    def __init__(self, max_steps: int = 10, max_cost: float = 1.0):
        self.max_steps = max_steps
        self.max_cost = max_cost
        self.current_cost = 0.0
        self.pii_patterns = [
            (r'\b\d{3}-\d{2}-\d{4}\b', '[SSN]'),  # SSN
            (r'\b\d{16}\b', '[CARD]'),  # Credit card
            (r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]'),  # Email
        ]
    
    def check_step_limit(self, current_step: int) -> bool:
        """Prevent infinite loops."""
        if current_step >= self.max_steps:
            raise ValueError(f"🛑 Safety: Step limit reached ({self.max_steps}). Possible infinite loop.")
        return True
    
    def check_cost_limit(self, additional_cost: float) -> bool:
        """Prevent cost explosions."""
        self.current_cost += additional_cost
        if self.current_cost >= self.max_cost:
            raise ValueError(f"🛑 Safety: Cost limit reached (${self.max_cost}). Current: ${self.current_cost:.4f}")
        return True
    
    def mask_pii(self, text: str) -> str:
        """Mask personally identifiable information."""
        masked = text
        for pattern, replacement in self.pii_patterns:
            masked = re.sub(pattern, replacement, masked)
        return masked
    
    def validate_citations(self, response: str, sources: List[str]) -> bool:
        """Ensure all factual claims have citations."""
        # Count citations in response
        citation_count = len(re.findall(r'\[Source: [^\]]+\]', response))
        
        # Heuristic: Should have at least one citation per source used
        if citation_count < len(set(sources)):
            print(f"⚠️  Warning: Low citation coverage ({citation_count} citations for {len(set(sources))} sources)")
            return False
        
        return True
    
    def reset_cost(self):
        """Reset cost counter (call per query)."""
        self.current_cost = 0.0

# Initialize guardrails
guardrails = SafetyGuardrails(max_steps=10, max_cost=1.0)
print("✅ Safety guardrails initialized")

✅ Safety guardrails initialized


### Test Guardrails

In [6]:
# Test PII masking
test_text = "My SSN is 123-45-6789 and email is john@example.com, card 4532015112830366"
masked = guardrails.mask_pii(test_text)
print("PII Masking Demo:")
print(f"Original: {test_text}")
print(f"Masked:   {masked}\n")

# Test citation validation
response_with_citations = "Order shipped on Jan 15 [Source: order_api]. Refund policy [Source: policy_kb]."
response_without_citations = "Order shipped on Jan 15. Refund takes 7 days."

print("Citation Validation Demo:")
print(f"With citations: {guardrails.validate_citations(response_with_citations, ['order_api', 'policy_kb'])}")
print(f"Without citations: {guardrails.validate_citations(response_without_citations, ['order_api', 'policy_kb'])}")

PII Masking Demo:
Original: My SSN is 123-45-6789 and email is john@example.com, card 4532015112830366
Masked:   My SSN is [SSN] and email is [EMAIL], card [CARD]

Citation Validation Demo:
With citations: True
⚠️  Warning: Low citation coverage (0 citations for 2 sources)
Without citations: False


---

## 4. Production-Ready Agent

### Putting It All Together

Now let's integrate:
- ✅ Caching
- ✅ Metrics
- ✅ Guardrails

With our agentic RAG from Part 2.

In [7]:
# Import agent from Part 2
from agentic_rag import build_agent, AgentState, ORDERS_DB

def production_agent_query(query: str) -> Dict[str, Any]:
    """
    Production-ready agent with caching, metrics, and safety.
    """
    start_time = time.time()
    guardrails.reset_cost()
    
    # Step 1: Check cache
    cached_response = cache.get(query, estimated_cost=0.05)
    if cached_response:
        end_time = time.time()
        return {
            "query": query,
            "response": cached_response,
            "from_cache": True,
            "latency": end_time - start_time,
            "cost": 0.0
        }
    
    # Step 2: Execute agent
    print(f"\n{'='*80}")
    print(f"❓ Query: {query}")
    print("="*80)
    
    agent = build_agent()
    initial_state = AgentState(
        query=query,
        messages=[{"role": "user", "content": query}]
    )
    
    # Run with safety checks
    tool_calls_log = []
    try:
        result = agent.invoke(initial_state)
        
        # Step 3: Safety validation
        guardrails.check_step_limit(result["steps"])
        guardrails.validate_citations(result["final_response"], result["sources"])
        
        # Mask PII in response
        safe_response = guardrails.mask_pii(result["final_response"])
        
        # Step 4: Log metrics
        end_time = time.time()
        total_latency = end_time - start_time
        estimated_cost = 0.001 * result["steps"] + 0.01  # Rough estimate
        
        # Count citations
        citation_count = len(re.findall(r'\[Source: [^\]]+\]', result["final_response"]))
        
        query_data = {
            "query": query,
            "steps": result["steps"],
            "total_latency": total_latency,
            "estimated_cost": estimated_cost,
            "citation_count": citation_count,
            "tool_calls": [
                {
                    "name": tr["tool"],
                    "args": tr["args"],
                    "success": "error" not in tr["result"],
                    "latency": 0.1  # Mock latency
                }
                for tr in result["tool_results"]
            ]
        }
        metrics.log_query(query_data)
        
        # Step 5: Store in cache
        cache.set(query, safe_response)
        
        # Step 6: Display results
        print(f"\n💬 Final Response:")
        print("-"*80)
        print(safe_response)
        print("-"*80)
        
        print(f"\n📊 Execution Stats:")
        print(f"   Latency: {total_latency:.3f}s")
        print(f"   Est. Cost: ${estimated_cost:.4f}")
        print(f"   Steps: {result['steps']}")
        print(f"   Tools Used: {len(result['tool_results'])}")
        print(f"   Citations: {citation_count}")
        
        return {
            "query": query,
            "response": safe_response,
            "from_cache": False,
            "latency": total_latency,
            "cost": estimated_cost,
            "steps": result["steps"],
            "citations": citation_count
        }
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return {
            "query": query,
            "error": str(e),
            "from_cache": False
        }

print("✅ Production agent ready")

✅ Production agent ready


---

## 5. Live Demo: Production Agent

In [8]:
# Test query 1 - Cache MISS
result1 = production_agent_query("Where is my order #12345?")

💾 [CACHE HIT - EXACT] Saved $0.0500


In [9]:
# Test query 2 - Cache HIT (exact match)
result2 = production_agent_query("Where is my order #12345?")

💾 [CACHE HIT - EXACT] Saved $0.0500


In [10]:
# Test query 3 - Cache HIT (semantic match)
result3 = production_agent_query("What's the status of order 12345?")


❓ Query: What's the status of order 12345?

🧠 Planning (Step 1/10)...
🔧 Executing 1 tool(s)...
   → check_order_status(order_id=12345)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------
Your order with ID **12345** has been successfully delivered on January 15, 2025 [Source: order_api]. Here are the details of your order:

- **Customer Email:** [EMAIL] [Source: order_api]
- **Items Ordered:**
  - Blue Cotton T-Shirt, Quantity: 2, Price: $29.99 each [Source: order_api]
  - Denim Jeans, Quantity: 1, Price: $59.99 [Source: order_api]

- **Total Amount:** $119.97 [Source: order_api]
- **Order Date:** January 10, 2025 [Source: order_api]
- **Shipping Date:** January 11, 2025 [Source: order_api]
- **Tracking Number:** 1Z999AA10123456784 [Source: order_api]

If you have any further questions or need assistance, please feel free to reach out. Thank you for shopping with

In [11]:
# Test query 4 - New query
result4 = production_agent_query("I want to return order #12345 because it's damaged. How much refund?")


❓ Query: I want to return order #12345 because it's damaged. How much refund?

🧠 Planning (Step 1/10)...
🔧 Executing 2 tool(s)...
   → check_order_status(order_id=12345)
   → calculate_refund(order_id=12345, reason=damaged)

🧠 Planning (Step 2/10)...

✍️  Generating final response with citations...

💬 Final Response:
--------------------------------------------------------------------------------
Your order #12345, which includes items such as "Blue Cotton T-Shirt" and "Denim Jeans," was delivered on January 15, 2025 [Source: order_api]. Since the items are damaged, you are eligible for a full refund of $119.97 [Source: refund_calculator]. There is no restocking fee for returns due to damage [Source: refund_calculator].

If you would like to proceed with the return, please let me know, and I can assist you further.
--------------------------------------------------------------------------------

📊 Execution Stats:
   Latency: 8.971s
   Est. Cost: $0.0120
   Steps: 2
   Tools Used: 2
 

---

## 6. Metrics Dashboard

In [12]:
# Show aggregated metrics
print("\n" + "="*80)
print("METRICS SUMMARY")
print("="*80)

print("\nAgent Performance:")
print(json.dumps(metrics.get_summary(), indent=2))

print("\nCache Performance:")
print(json.dumps(cache.get_stats(), indent=2))

# Show detailed trace for last query
metrics.print_trace(-1)


METRICS SUMMARY

Agent Performance:
{
  "total_queries": 2,
  "avg_latency_sec": 8.907,
  "avg_cost_usd": 0.012,
  "avg_steps": 2.0,
  "tool_stats": {}
}

Cache Performance:
{
  "exact_hits": 3,
  "semantic_hits": 0,
  "misses": 4,
  "total_cost_saved": 0.15000000000000002,
  "total_queries": 7,
  "hit_rate": 0.42857142857142855,
  "cache_size": 3
}

EXECUTION TRACE
Query: I want to return order #12345 because it's damaged. How much refund?
Timestamp: 2025-11-05T12:34:02.721129
Total Latency: 8.971s
Estimated Cost: $0.0120
Steps: 2
Citations: 3

Tool Calls (2)
  1. check_order_status({'order_id': '12345'})
     → Latency: 0.100s
     → Status: ✅ Success
  2. calculate_refund({'order_id': '12345', 'reason': 'damaged'})
     → Latency: 0.100s
     → Status: ✅ Success


---

## 7. FastAPI Deployment Wrapper

### Production API Endpoint

Here's how to deploy as a REST API:

In [13]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.responses import JSONResponse

app = FastAPI(title="Agentic RAG API", version="1.0.0")

class QueryRequest(BaseModel):
    query: str
    user_id: Optional[str] = None

class QueryResponse(BaseModel):
    query: str
    response: str
    from_cache: bool
    latency_sec: float
    cost_usd: float
    metadata: Dict[str, Any]

@app.post("/v1/query", response_model=QueryResponse)
async def query_agent(request: QueryRequest):
    """
    Query the agentic RAG system.
    
    - **query**: The user's question
    - **user_id**: Optional user identifier for rate limiting
    """
    try:
        result = production_agent_query(request.query)
        
        if "error" in result:
            raise HTTPException(status_code=500, detail=result["error"])
        
        return QueryResponse(
            query=result["query"],
            response=result["response"],
            from_cache=result["from_cache"],
            latency_sec=result["latency"],
            cost_usd=result.get("cost", 0.0),
            metadata={
                "steps": result.get("steps", 0),
                "citations": result.get("citations", 0)
            }
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/v1/metrics")
async def get_metrics():
    """Get agent performance metrics."""
    return JSONResponse(content={
        "agent": metrics.get_summary(),
        "cache": cache.get_stats()
    })

@app.get("/health")
async def health_check():
    """Health check endpoint."""
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}

print("✅ FastAPI app configured")
print("\nTo run the API server:")
print("  python -m uvicorn production_agent:app --host 0.0.0.0 --port 8000")
print("\nThen test with:")
print("  curl -X POST http://localhost:8000/v1/query \\")
print("       -H 'Content-Type: application/json' \\")
print("       -d '{\"query\": \"Where is my order #12345?\"}'")

✅ FastAPI app configured

To run the API server:
  python -m uvicorn production_agent:app --host 0.0.0.0 --port 8000

Then test with:
  curl -X POST http://localhost:8000/v1/query \
       -H 'Content-Type: application/json' \
       -d '{"query": "Where is my order #12345?"}'


---

## 8. Cost Analysis

Let's analyze the cost savings from our optimizations:

In [14]:
def cost_analysis():
    """Compare costs: baseline vs production."""
    
    print("\n" + "="*80)
    print("COST ANALYSIS")
    print("="*80)
    
    # Assumptions
    queries_per_day = 1000
    cache_hit_rate = cache.get_stats().get("hit_rate", 0)
    avg_cost_per_query = 0.05  # Without cache
    
    # Without caching
    cost_without_cache_daily = queries_per_day * avg_cost_per_query
    cost_without_cache_monthly = cost_without_cache_daily * 30
    
    # With caching
    cached_queries = queries_per_day * cache_hit_rate
    uncached_queries = queries_per_day * (1 - cache_hit_rate)
    cost_with_cache_daily = uncached_queries * avg_cost_per_query
    cost_with_cache_monthly = cost_with_cache_daily * 30
    
    savings_daily = cost_without_cache_daily - cost_with_cache_daily
    savings_monthly = cost_without_cache_monthly - cost_with_cache_monthly
    savings_pct = (savings_monthly / cost_without_cache_monthly) * 100 if cost_without_cache_monthly > 0 else 0
    
    print(f"\nAssumptions:")
    print(f"  Queries per day: {queries_per_day:,}")
    print(f"  Cache hit rate: {cache_hit_rate:.1%}")
    print(f"  Avg cost per query: ${avg_cost_per_query:.4f}")
    
    print(f"\nWithout Caching:")
    print(f"  Daily cost: ${cost_without_cache_daily:.2f}")
    print(f"  Monthly cost: ${cost_without_cache_monthly:.2f}")
    
    print(f"\nWith Caching:")
    print(f"  Cached queries/day: {cached_queries:.0f}")
    print(f"  Uncached queries/day: {uncached_queries:.0f}")
    print(f"  Daily cost: ${cost_with_cache_daily:.2f}")
    print(f"  Monthly cost: ${cost_with_cache_monthly:.2f}")
    
    print(f"\n💰 Savings:")
    print(f"  Daily: ${savings_daily:.2f}")
    print(f"  Monthly: ${savings_monthly:.2f}")
    print(f"  Percentage: {savings_pct:.1f}%")
    
    print("\n" + "="*80)

cost_analysis()


COST ANALYSIS

Assumptions:
  Queries per day: 1,000
  Cache hit rate: 42.9%
  Avg cost per query: $0.0500

Without Caching:
  Daily cost: $50.00
  Monthly cost: $1500.00

With Caching:
  Cached queries/day: 429
  Uncached queries/day: 571
  Daily cost: $28.57
  Monthly cost: $857.14

💰 Savings:
  Daily: $21.43
  Monthly: $642.86
  Percentage: 42.9%



---

## Summary: Production Checklist

### What We Built

1. **Caching Layer**
   - Exact match (hash-based)
   - Semantic similarity (embedding-based)
   - TTL expiration
   - Hit rate tracking

2. **Observability**
   - Latency tracking (total + per-tool)
   - Cost estimation
   - Citation coverage
   - Execution traces

3. **Safety Guardrails**
   - Step limits (prevent infinite loops)
   - Cost limits (prevent explosions)
   - PII masking
   - Citation validation

4. **Production API**
   - FastAPI wrapper
   - Health checks
   - Metrics endpoint
   - Error handling

---

### Next Steps

**To deploy this agent**:

1. **Scaling**:
   - Use Redis for distributed caching
   - Add rate limiting per user
   - Implement request queuing

2. **Monitoring**:
   - Integrate OpenTelemetry for tracing
   - Set up Prometheus metrics
   - Add alerting (Sentry, PagerDuty)

3. **Evaluation**:
   - Use RAGAS for quality metrics
   - A/B test different prompts
   - Collect user feedback

4. **Cost Optimization**:
   - Use cheaper models for routing
   - Batch embedding calls
   - Implement response streaming

---

### 🎓 Key Takeaways

1. **Caching is critical** - Can save 50-80% on costs at scale
2. **Observability enables iteration** - You can't improve what you don't measure
3. **Safety first** - Guardrails prevent production disasters
4. **Start simple, add complexity gradually** - Don't over-engineer

---

**You now have a production-ready agentic RAG system!** 🎉

From basic retrieval (Part 1) → Agentic reasoning (Part 2) → Production deployment (Part 3).